## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-12-18-58-01 +0000,nyt,Iran Says It Is Prepared for War but Ready to ...,https://www.nytimes.com/2026/01/12/world/middl...
1,2026-01-12-18-56-38 +0000,nyt,Trump’s Envoy to India Offers Hope Amid a Stra...,https://www.nytimes.com/2026/01/12/world/asia/...
2,2026-01-12-18-55-41 +0000,nypost,Biden admin likely paid out $84.6M in error fo...,https://nypost.com/2026/01/12/us-news/biden-ad...
3,2026-01-12-18-54-00 +0000,wsj,Thousands of New York City Nurses Go On Strike,https://www.wsj.com/health/healthcare/thousand...
4,2026-01-12-18-53-40 +0000,nypost,"MTA probing AI to identify weapons, monitor sa...",https://nypost.com/2026/01/12/us-news/mta-prob...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,43
0,iran,23
62,powell,20
331,fed,18
332,chair,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
170,2026-01-12-10-30-00 +0000,wsj,"For years, Fed Chair Jerome Powell responded t...",https://www.wsj.com/economy/central-banking/fo...,128
78,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...,120
195,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...,119
152,2026-01-12-11-32-54 +0000,nypost,Trump addresses bombshell Jerome Powell invest...,https://nypost.com/2026/01/12/us-news/trump-sa...,114
146,2026-01-12-12-03-00 +0000,wsj,How the U.S. and Iran went back to the brink o...,https://www.wsj.com/politics/policy/how-the-u-...,109


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
170,128,2026-01-12-10-30-00 +0000,wsj,"For years, Fed Chair Jerome Powell responded t...",https://www.wsj.com/economy/central-banking/fo...
35,72,2026-01-12-17-59-00 +0000,wsj,Mary Peltola will challenge GOP incumbent Sen....,https://www.wsj.com/politics/elections/mary-pe...
127,65,2026-01-12-13-24-31 +0000,cbc,Swiss court keeps bar owner in custody after d...,https://www.cbc.ca/news/world/swiss-court-bar-...
198,53,2026-01-12-05-36-07 +0000,bbc,Trump mulls 'very strong' military options as ...,https://www.bbc.com/news/articles/czdqy3rmp78o...
108,44,2026-01-12-14-33-37 +0000,cbc,"Malaysia, Indonesia become 1st countries to bl...",https://www.cbc.ca/news/business/malaysia-indo...
33,38,2026-01-12-18-03-44 +0000,nypost,Meta warns Australia’s under-16 social media b...,https://nypost.com/2026/01/12/business/meta-wa...
2,36,2026-01-12-18-55-41 +0000,nypost,Biden admin likely paid out $84.6M in error fo...,https://nypost.com/2026/01/12/us-news/biden-ad...
151,35,2026-01-12-11-50-09 +0000,nypost,Gut-wrenching footage shows grieving Iranian f...,https://nypost.com/2026/01/12/world-news/foota...
70,34,2026-01-12-16-51-12 +0000,nypost,Major NYC hospital accuses nurses’ union of pr...,https://nypost.com/2026/01/12/us-news/major-ny...
195,33,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
